In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
articles = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')
transactions = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
customer  = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv')

In [ ]:
articles['article_id'] = articles['article_id'].astype(str)
transactions['article_id'] = transactions['article_id'].astype(str)
articles['article_id'] = articles['article_id'].apply(lambda x : x.zfill(10))
transactions['article_id'] = transactions['article_id'].apply(lambda x : x.zfill(10))

In [ ]:
customer.nunique()

In [ ]:
100*customer.isnull().sum()/customer.shape[0]

In [ ]:
100*customer['fashion_news_frequency'].value_counts()/customer.shape[0]

In [ ]:
100*customer['club_member_status'].value_counts()/customer.shape[0]

In [ ]:
customer.drop(labels = ['FN','Active','fashion_news_frequency'], inplace = True, axis =1)

In [ ]:
# Looks Right Skewed
customer['age'].hist()

In [ ]:
customer['age'] = customer['age'].fillna(customer['age'].median())

In [ ]:
# imputing 

customer['club_member_status'] = customer['club_member_status'].fillna(customer['club_member_status'].mode().values[0])

In [ ]:
def make_buckets(x):
    if x >= 16 and x<=24:
        return 'Youth'
    elif x> 24 and x <= 40:
        return 'Young Adults'
    elif x>40 and x <= 64:
        return 'Middle Age Adults'
    elif x>64:
        return 'Seniors'
        

In [ ]:
customer['age_bucket'] = customer['age'].apply(lambda x : make_buckets(x))

In [ ]:
transactions.drop(labels = ['sales_channel_id'], inplace = True, axis = 1)

In [ ]:
merged_data1 = pd.merge(left = transactions, right = customer , how = 'left', on = 'customer_id')

In [ ]:

# merged_data1.head()

In [ ]:
# merged_data1.isnull().sum()

In [ ]:
# del(customer)
del(transactions)

In [ ]:
# articles.isnull().sum()

In [ ]:
# merged_data1.shape
# import gc
# gc.collect()

In [ ]:
MD2 = pd.merge(left = merged_data1, right = articles[['article_id','prod_name','index_group_name','graphical_appearance_name','index_name','section_name','garment_group_name']], on = 'article_id', how = 'left')

In [ ]:
MD2.shape

In [ ]:
# MD2.isnull().sum()

In [ ]:
del(merged_data1)

In [ ]:
# MD2.head()

In [ ]:
# Sales and Spending Across Age groups

In [ ]:
Sales_age_group = MD2.groupby(['age_bucket'])['price'].sum().reset_index()

In [ ]:
sns.barplot(x = 'age_bucket',
            y = 'price',
            data = Sales_age_group.sort_values(by = ['price'],ascending = False))
plt.xticks(rotation=90)

In [ ]:
transations_per_group = MD2['age_bucket'].value_counts().reset_index()
transations_per_group.columns  = ['age_bucket', 'orders']

In [ ]:
sns.barplot(x = 'age_bucket',
            y = 'orders',
            data = transations_per_group)
plt.xticks(rotation=90)

In [ ]:
# Clearly Adults i.e age group between 24 and 64 spend more as they belong to earning class

# Popular Products Categories

In [ ]:
prod_name = MD2['prod_name'].value_counts().reset_index()
prod_name.columns  = ['prod_name','orders']

In [ ]:
sns.barplot(x = 'prod_name',
            y = 'orders',
            data = prod_name.head(10))
plt.xticks(rotation=90)

In [ ]:
index_group_name = MD2['index_group_name'].value_counts().reset_index()
index_group_name.columns  = ['index_group_name','orders']


sns.barplot(x = 'index_group_name',
            y = 'orders',
            data = index_group_name.head(10))
plt.xticks(rotation=90)

In [ ]:
# Popular designs
graphical_appearance_name = MD2['graphical_appearance_name'].value_counts().reset_index()
graphical_appearance_name.columns  = ['graphical_appearance_name','orders']


sns.barplot(x = 'graphical_appearance_name',
            y = 'orders',
            data = graphical_appearance_name.head(10))
plt.xticks(rotation=90)

In [ ]:

index_name = MD2['index_name'].value_counts().reset_index()
index_name.columns  = ['index_name','orders']


sns.barplot(x = 'index_name',
            y = 'orders',
            data = index_name)
plt.xticks(rotation=90)

In [ ]:
# Popular Sections

MD2['section_name'].value_counts()


section_name = MD2['section_name'].value_counts().reset_index()
section_name.columns  = ['section_name','orders']


sns.barplot(x = 'section_name',
            y = 'orders',
            data = section_name.head(10))
plt.xticks(rotation=90)

# Clearly H&M is more popular among female customers

In [ ]:
garment_group_name = MD2['garment_group_name'].value_counts().reset_index()
garment_group_name.columns  = ['garment_group_name','orders']


sns.barplot(x = 'garment_group_name',
            y = 'orders',
            data = garment_group_name.head(10))
plt.xticks(rotation=90)

# Lets see Age Group preferences

In [ ]:
Age_pref_sec = MD2.groupby(['age_bucket','section_name'])['article_id'].count().reset_index()
Age_pref_sec.sort_values(by = ['age_bucket','article_id'], ascending = False, inplace = True)

In [ ]:
# Top 4 Sections per user group
Age_pref_sec.groupby(['age_bucket']).head(4).reset_index(drop=True)

# We can clearly see 'Womens Everyday Collection' is popular among all age categories

In [ ]:
# getting Customer Preferred Sections

In [ ]:
cust_pref_sec = MD2.groupby(['customer_id','section_name'])['article_id'].count().reset_index()

In [ ]:
# cust_pref_sec[cust_pref_sec['customer_id']=='00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657'].sort_values(by = ['article_id'], ascen)

In [ ]:
cust_pref_sec.sort_values(by =['customer_id','article_id'], ascending = False, inplace = True)

In [ ]:
cust_pref_sec.reset_index(drop=True, inplace = True)

In [ ]:
cust_no_sec = cust_pref_sec.groupby(['customer_id'])['section_name'].count().reset_index()

In [ ]:
cust_no_sec.sort_values(by = 'section_name', inplace = True)

In [ ]:
Customer_list_1 = cust_no_sec[cust_no_sec.section_name >2 ]['customer_id'].unique().tolist()

In [ ]:
Customer_list_2 = cust_no_sec[cust_no_sec.section_name <=2 ]['customer_id'].unique().tolist()

# Recommendation Generation

In [ ]:
# Recommend Customer_list_2 products based on their age_group preference
# Recommend Customer_list_1 products based on their sections (Top3)

In [ ]:
cust_pref_sec_list1 = cust_pref_sec[cust_pref_sec['customer_id'].isin(Customer_list_1)].reset_index(drop=True)

In [ ]:
cust_pref_sec_list1 = cust_pref_sec_list1.groupby('customer_id').head(3)

In [ ]:
pop_art = MD2.groupby(['section_name','article_id'])['customer_id'].count().reset_index()

In [ ]:
pop_art.sort_values(by = ['section_name','customer_id'], ascending = False, inplace = True)

In [ ]:
pop_art = pop_art.groupby('section_name').head(4).reset_index(drop=True)

In [ ]:
cust_pref_sec_list1

In [ ]:
pop_art.drop(labels = 'customer_id', axis =1, inplace = True)

In [ ]:
Age_pref_sec = Age_pref_sec.groupby(['age_bucket']).head(3).reset_index(drop=True)
Age_pref_sec.reset_index(drop=True, inplace = True)

In [ ]:
Age_pref_sec.drop(labels = 'article_id', axis =1, inplace = True)

In [ ]:
Age_pref_sec

In [ ]:
pop_art

In [ ]:
pop_art_recom = []
for i in tqdm(pop_art.section_name.unique().tolist()):
    
    artcle = pop_art[pop_art.section_name == i]['article_id'].unique().tolist()
    artcle = [str(i) for i in artcle]
    
    recommend = ' '.join(artcle)
    
    pop_art_recom.append({"section_name":i,
                          "recommend": recommend
                         })
    
    
    
    

In [ ]:
recom_sec = pd.DataFrame(pop_art_recom)

In [ ]:
recom_sec.head()

In [ ]:
age_recom = []
for i in Age_pref_sec.age_bucket.unique().tolist():
    
    section = Age_pref_sec[Age_pref_sec.age_bucket == i]['section_name'].unique().tolist()
    
    
    rec2 = recom_sec[recom_sec.section_name.isin(section)]['recommend'].unique().tolist()
    
    rec3 = ' '.join(rec2)
    
    age_recom.append({"age_bucket":i,
                          "recommend": rec3
                         })
    

In [ ]:
age_recoom = pd.DataFrame(age_recom)

In [ ]:
Submission = customer[['customer_id', 'age_bucket']]

In [ ]:
Submission.nunique()

In [ ]:
cust_pref_sec_list1.drop(labels = ['article_id'],axis = 1, inplace = True)

In [ ]:
recom_type1_merge = pd.merge(left = cust_pref_sec_list1, right = recom_sec, on = 'section_name', how = 'left')

In [ ]:
recom_type1_merge['prediction'] = recom_type1_merge.groupby(['customer_id'])['recommend'].transform(
                                              lambda x: ' '.join(x))

In [ ]:
recom_type1_merge.drop_duplicates(subset = ['customer_id'], keep='first', inplace =True)

In [ ]:
recom_type1_merge.drop(labels = ['section_name','recommend'], axis =1, inplace = True)

In [ ]:
rec_merge2= Submission[~Submission.customer_id.isin(recom_type1_merge.customer_id.unique().tolist())]

In [ ]:
rec_merge2.reset_index(drop=True, inplace=True)

In [ ]:
rec_merge2.head()

In [ ]:
age_recoom.head()

In [ ]:
rec_merge2 = pd.merge(left =rec_merge2, right = age_recoom, on = 'age_bucket', how ='left')

In [ ]:
rec_merge2.drop(labels = 'age_bucket', axis=1, inplace = True)

In [ ]:
rec_merge2.rename(columns = {'recommend':'prediction'}, inplace = True)

In [ ]:
rec_merge2.shape[0]+recom_type1_merge.shape[0]

In [ ]:
final_submission = pd.DataFrame()
final_submission = final_submission.append([recom_type1_merge,rec_merge2])

In [ ]:
final_submission.reset_index(drop=True, inplace = True)

In [ ]:
final_submission.head(2)['prediction'][0]

In [ ]:
final_submission.head()

# Upvote if u liked it!